# Unvariate Kalman filter

# Purpose
* implementation of 1D linear kalman filter inspired by: [Kalman-and-Bayesian-Filters-in-Python/blob/master/04-One-Dimensional-Kalman-Filters](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/04-One-Dimensional-Kalman-Filters.ipynb)

# Methodology
* Implement a kalman filter to chase a ship at steady velocity

# Setup

In [ ]:
# %load imports.py

%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import filterpy.stats as stats

In [ ]:
from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: '𝒩(μ={:.3f}, 𝜎²={:.3f})'.format(s[0], s[1])

In [ ]:
gaussian(mean=0, var=1)

What is the sum of two Gaussians? In the last chapter I proved that:

$$\begin{gathered}
\mu = \mu_1 + \mu_2 \\
\sigma^2 = \sigma^2_1 + \sigma^2_2
\end{gathered}$$

In [ ]:
def predict(pos, movement):
    return gaussian(pos.mean + movement.mean, pos.var + movement.var)

In [ ]:
def gaussian_multiply(g1, g2):
    mean = (g1.var * g2.mean + g2.var * g1.mean) / (g1.var + g2.var)
    variance = (g1.var * g2.var) / (g1.var + g2.var)
    return gaussian(mean, variance)

def update(prior, likelihood):
    posterior = gaussian_multiply(likelihood, prior)
    return posterior

In [ ]:
z = gaussian(10., 1.)  # Gaussian N(10, 1)

product = gaussian_multiply(z, z)

xs = np.arange(5, 15, 0.1)
ys = [stats.gaussian(x, z.mean, z.var) for x in xs]
plt.plot(xs, ys, label='$\mathcal{N}(10,1)$')

ys = [stats.gaussian(x, product.mean, product.var) for x in xs]
plt.plot(xs, ys, label='$\mathcal{N}(10,1) \\times \mathcal{N}(10,1)$', ls='--')
plt.legend()
print(product)

## Kalman filtering

In [ ]:
process_var = 0.5**2.
sensor_var = 5**2
x = gaussian(0., sensor_var**2)
v_ = 5.5
process_var=2.0
N=40
t_ = np.linspace(0,10,N)
dt = t_[1]-t_[0]

x_real = v_*t_
zs = x_real + np.random.normal(scale=5, size=N)

process_model = gaussian(v_*dt, process_var)

xs, priors, np.zeros((N, 2)), np.zeros((N, 2)), np.zeros((N, 2))

for i, z in enumerate(zs):
    
    prior = predict(x, process_model)    
    z_ = gaussian(z, sensor_var)
    x = update(prior, z_)
    
    # Save
    priors[i] = prior
    xs[i] = x

In [ ]:
fig,ax=plt.subplots()

ax.plot(t_, x_real, '-', label='real')
ax.plot(t_, zs, 'o', label='measurement')
ax.plot(t_, xs[:,0], '-', label='filter')
ax.plot(t_, priors[:,0], '--', label='prior')
ax.legend()


fig,ax=plt.subplots()
ax.plot([t_[0],t_[-1]], [sensor_var, sensor_var], label='measurement')
ax.plot(t_, xs[:,1], label='filter')
ax.plot(t_, priors[:,1], label='prior')
ax.set_title('Variances')
ax.set_ylim(0,50)
ax.legend()



In [ ]:
for x in xs[0::5]:

    mu = x[0]
    var = x[1]
    sigma = np.sqrt(var)
        
    xs_ = np.linspace(mu-3*sigma, mu+3*sigma,100)
    
    ys_ = [stats.gaussian(x_, mu, var) for x_ in xs_]
    plt.plot(xs_, ys_, ls='--')


In [ ]:
for x in priors[0::5]:

    mu = x[0]
    var = x[1]
    sigma = np.sqrt(var)
        
    xs_ = np.linspace(mu-3*sigma, mu+3*sigma,100)
    
    ys_ = [stats.gaussian(x_, mu, var) for x_ in xs_]
    plt.plot(xs_, ys_, ls='--')
